In [57]:
# import libs

import numpy as np
import Laser
import gCode
import random
import Printer


# Important definition
positive angle in CCW direction

### Units
Distances: mm  
Angles: degrees

# Define functions

In [58]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    tmp = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        tmp.append(line)
    
    return np.array(tmp)
    

In [59]:
def parseCoord(gCodeObj,linestring):
    line = linestring
    X = None
    Y = None
    Z = None
    E = None
    F = None
    move = False

    # check the command of the gcode line (string until first space)
    command = line.split(' ')[0]

    # check if line is ;LAYER:1 and gCodeObj.firstLayerCheck is False
    if line.startswith(';LAYER:1') and not gCodeObj.firstLayerCheck:
        gCodeObj.updateFirstLayerCheck(True)
        print("First layer check is True")

    # check if firstLayerCheck is True
    if gCodeObj.firstLayerCheck:
        # if command == G1 or G0 than move = True
        if command == 'G1' or command == 'G0':
            move = True
        if command == 'G1':
            gCodeObj.updateNumberOfG1()
        # check if line contains ; which means it is a comment and skip line
        if not ';' in line:
            # parse the X Y Z coords from string to float
            if 'X' in line:
                X = float(line.split('X')[-1].split(' ')[0])
            elif 'X' not in line:
                X = None
            if 'Y' in line:
                Y = float(line.split('Y')[-1].split(' ')[0])
            elif 'Y' not in line:
                Y = None
            if 'Z' in line:
                Z = float(line.split('Z')[-1].split(' ')[0])
            elif 'Z' not in line:
                Z = None
            if 'E' in line:
                E = float(line.split('E')[-1].split(' ')[0])
            elif 'E' not in line:
                E = None
            if 'F' in line:
                F = float(line.split('F')[-1].split(' ')[0])
                gCodeObj.updateLastF(F)
            elif 'F' not in line:
                F = gCodeObj.getLastF()
    else:
        X = None
        Y = None
        Z = None
        E = None
        F = None

    return move, X, Y, Z, E, F, command, linestring


In [60]:
# calculate the distance between two points
def calcDistance(x0, y0, x1, y1):
    return np.sqrt((x1-x0)**2 + (y1-y0)**2)


In [61]:
# calculate angle between two points (atan2)
def calcAngle(x0, y0, x1, y1):
    newAngle = np.arctan2((y1-y0),(x1-x0))
   
    # convert to degrees
    newAngle = np.rad2deg(newAngle)
    
    # normalize angle
    #newAngle = np.mod(newAngle, 360)
    
    return newAngle

In [62]:
def findClosestAngle(targetAngle, LaserObj):
    currentAngles = LaserObj.getNormAngles()
    
    # find the closest angle in the list of angles
    closestAngleIndx = np.argmin(np.abs(currentAngles - targetAngle))
    closestAngle = currentAngles[closestAngleIndx]

    # calculte delta bewteen target and closest angle
    deltaAngle = targetAngle - closestAngle

    # update the angle of the laser
    #LaserObj.updateAngles(deltaAngle)

    return closestAngleIndx, deltaAngle

    

In [63]:

def convertAngleToSteps(angle):
    return angle


In [64]:
def postProcessGcode(gCodeArray, threshold, LaserObj, newFilePath, gCodeObj):

    for i in range(gCodeArray.shape[0]-2):              # Die letzte Zeile wird nicht mehr betrachtet
        move0, X0, Y0, Z0, E0, F0, command0, line0 = parseCoord(gCodeObj, gCodeArray[i])
        move1, X1, Y1, Z1, E1, F1, command1, line1 = parseCoord(gCodeObj, gCodeArray[i+1])


        # update lastPos if move1 is False and move0 is True
        if move0 and not move1:
            gCodeObj.updateCurrentPos([X0, Y0, Z0])

        # clear Extruder for testing
        if(0):
            # copy line0 until reaches E 
            line0 = line0.split('E')[0] + '\n'

        # check if both lines are moves
        if move0 and move1:
            # check if the distance between the two points is larger than the threshold
        
            #print("i:", i+1, "move0: ", move0, "move1: ", move1, "X0: ", X0, "Y0: ", Y0, "Z0: ", Z0, "command0: ", command0, "line0: ", line0, "X1: ", X1, "Y1: ", Y1, "Z1: ", Z1, "command1: ", command1, "line1: ", line1)
            dist = calcDistance(X0, Y0, X1, Y1)
            gCodeObj.updateTotalLengthOfG1(dist)

            # calculate the angle between the two points
            targetAngle = calcAngle(X0, Y0, X1, Y1)

            # update the last angle
            gCodeObj.updateLastAngle(targetAngle)

            # war vorher in der if schleife
            # noralize the target angle
            targetAngle = np.mod(targetAngle, 360)

            # find the closest angle to the target angle
            closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, LaserObj)

            # update the angle of the laser
            LaserObj.updateAngles(deltaAngle)

            # update absolute total laser movement
            LaserObj.updateAbsLaserMovement(deltaAngle)

            # convert the angle to steps
            steps = convertAngleToSteps(deltaAngle)

            # corrected laser connection
            laserIndx = closestAngleIndx + 2

            if dist > threshold:

                
                # add new gcode line
                    # rotate the laser by delta angle
                        # for relative axis movement of laser introduce a G91
                        # reset the relative axis movement with G90

                tmpAngle = LaserObj.getAngles()[closestAngleIndx]

                

                # activate relative movement, rotate laser, deactivate relative movement
                Line = "G91 \nG0 A" + str(steps) + " \nG90 \n"
                
                # turn laser on
                Line += "M106 P" + str(laserIndx) + " S1 \n"

                # move X Y pos to next point and extrude filament
                Line += str(command1) + " F"+ str(F1) +" X" + str(X1) + " Y" + str(Y1) + " E" + str(E1) + " \n"
                # turn laser off
                Line += "M106 P" + str(laserIndx) + " S0 \n"
                
                # append new lines to newFilePath
                with open(newFilePath, 'a') as f:
                    f.write(Line)

            elif (closestAngleIndx == LaserObj.getLastActivatedLaser()):
                # on and off toggeling of laser deactivated

                # activate relative movement, rotate laser, deactivate relative movement
                Line = "G91 \nG0 A" + str(steps) + " \nG90 \n"

                # activate Laser
                Line += "M106 P" + str(laserIndx) + " S1 \n"

                # move X Y pos to next point and extrude filament
                Line += str(command1) + " F"+ str(F1) +" X" + str(X1) + " Y" + str(Y1) + " E" + str(E1) + " \n"

                # check if i+1 and i+2 have same laser as well, else turn off laser
                move2, X2, Y2, Z2, E2, F2, command2, line2 = parseCoord(gCodeObj, gCodeArray[i+2])

                # calculate the angle between the two points
                targetAngle2 = calcAngle(X1, Y1, X2, Y2)
                
                # noralize the target angle
                targetAngle = np.mod(targetAngle2, 360)

                # find the closest angle to the target angle
                closestAngleIndx, deltaAngle = findClosestAngle(targetAngle2, LaserObj)

                # check if next line has not the same laser
                if (closestAngleIndx != LaserObj.getLastActivatedLaser()):
                    # deactivate Laser
                    Line += "M106 P" + str(closestAngleIndx) + " S0 \n"

                # write line to file
                with open(newFilePath, 'a') as f:
                    f.write(Line)
                

            # if dist is smaller than threshold
            else:
                with open(newFilePath, 'a') as f:
                    f.write(line1)
                    f.close()
                    print("other line appeneded: ", line1)

            # save last used laser
            LaserObj.updateLastActivatedLaser(closestAngleIndx)
                        
        else:
            # append line0 to newFilePath
            with open(newFilePath, 'a') as f:
                f.write(line1)
                f.close()
                print("other line appeneded: ", line1)
            
        

    return True

# Test functions

In [65]:
# test calcAngle function
if (0):
    print(calcAngle(0,0,0,0))
    print(calcAngle(0,0,1,-1))
    print(calcAngle(0,0,-1,-1))
    print(calcAngle(0,0,-1,1))


In [66]:
# test the parseCoord function
if(0):

# read the gcode file
    gcode = read_gcode_file("GCODE.txt")

    # create empty lists for the coordinates
    X = []
    Y = []
    Z = []
    cmd = []
    i = 0
    # loop over all lines in the gcode file
    for line in gcode:
        #print(i)
        #i += 1
        # parse the coordinates from the line
        moveBool, x, y, z, command, linestring = parseCoord(line)
        # append the coordinates to the lists
        X.append(x)
        Y.append(y)
        Z.append(z)
        cmd.append(command)

# print first 10 elements of X Y Z
j = 50
if(0):
    for i in range(j):
        print(f"X: {X[i]} Y: {Y[i]} Z: {Z[i]} cmd: {cmd[i]}")

# Main 

In [67]:
# initiate Printer object, for duration calculation of process
Prusa = Printer.Printer(velocity_trans=100, velocity_angular=100)

# read the gcode file
gcode = read_gcode_file("Input/PI3MK3M_Circle_50mm.gcode")
gCodeObj = gCode.gCode("gCode")

# print shape of gcode
print(gcode.shape)

# create laser object
numberOfLaser = 4
laser = Laser.Laser(numberOfLaser)

# threshold for the distance between two points 
minDistance = 2

# check index in Output/index.txt
with open("Output/index.txt", 'r') as f:
    index = int(f.read())
    f.close()

# create new file
newFilePath = "Output/GCODE_postprocessed_" + str(index) + ".gcode"

# iterate index in Output/index.txt
with open("Output/index.txt", 'w') as f:
    f.write(str(index+1))
    f.close()

# post process the gcode
print(postProcessGcode(gcode, minDistance, laser, newFilePath, gCodeObj))


(257,)
other line appeneded:  G0 X125.611 Y129.392

other line appeneded:  ;TIME_ELAPSED:188.414124

First layer check is True
other line appeneded:  ;LAYER:1

other line appeneded:  M106 S255

other line appeneded:  ;TYPE:WALL-INNER

other line appeneded:  ;MESH:Circle_50mm.stl

other line appeneded:  G1 F2700 X126.43 Y129.357 E175.33461

other line appeneded:  G1 X127.247 Y129.295 E175.35505

other line appeneded:  G1 X128.825 Y129.096 E175.39473

other line appeneded:  G1 X130.482 Y128.775 E175.43684

other line appeneded:  G1 X132.1 Y128.344 E175.47862

other line appeneded:  G1 X133.649 Y127.816 E175.51944

other line appeneded:  G1 X135.133 Y127.195 E175.55958

other line appeneded:  G1 X136.602 Y126.463 E175.60052

other line appeneded:  G1 X138.064 Y125.609 E175.64276

other line appeneded:  G1 X139.374 Y124.714 E175.68234

other line appeneded:  G1 X140.666 Y123.705 E175.72324

other line appeneded:  G1 X141.885 Y122.61 E175.76412

other line appeneded:  G1 X143.03 Y121.436 E1

In [68]:
print(gCodeObj.firstLayerCheck)
print(laser.absLaserMovement)
print("Total lenght of G1 commands:\t " + str(gCodeObj.getTotalLengthOfG1()) + " mm")
print("Total angle turned: \t" + str(laser.getAbsLaserMovement()))
print("Total number of G1 commands: \t\t" + str(gCodeObj.getNumberOfG1()))



True
-2.150783935839115
Total lenght of G1 commands:	 202.03857291599942 mm
Total angle turned: 	-2.150783935839115
Total number of G1 commands: 		661
